# Problem 1

We saw how to solve TSPs in this module and in particular presented two approaches to encode a TSP as an integer linear program. In this problem, we will ask you to adapt the TSP solution to the related problem of $k$ Travelling Salespeople Problem ($k$-TSP).

Let $G$ be a complete graph with $n$ vertices that we will label $0, \ldots, n-1$ (keeping Python array indexing in mind). Our costs are specified using a matrix $C$ wherein $C_{i,j}$ is the cost of the edge from vertex $i$ to $j$ for $i \not= j$.

In this problem, we have $k \geq 1$ salespeople who must  start from vertex $0$ of the graph (presumably the location of the sales office) and together visit every location in the graph,  each returning back to vertex $0$. Each location must be visited exactly once by some salesperson in the team. Therefore, other than vertex $0$ (the start/end vertex of every salesperson's tour), no two salesperson tours have a vertex in common. Notice that for $k=1$, this is just the regular TSP problem we have studied. 

Also, all $k$ salespeople must be employed in the tour. In other words, if we have $k=3$ then each salesperson must start at $0$ and visit a sequence of one or more vertices and come back to $0$. No salesperson can be "idle".

## Example-1

Consider a graph with $5$ nodes and the following cost matrix:

$$ \begin{array}{c|ccccc}
  \text{Vertices} & 0 & 1 & 2 & 3 & 4 \\ 
   \hline
 0 & - & 3 & 4 & 3 & 5 \\ 
 1 & 1 & - & 2 & 4 & 1 \\ 
 2 & 2 & 1 & - & 5 & 4 \\ 
 3 & 1 & 1 & 5 & - & 4 \\ 
 4 & 2 & 1 & 3 & 5 & - \\ 
 \end{array}$$
 
 For instance $C_{2,3}$ the cost of edge from vertex $2$ to $3$ is $5$. The $-$ in the diagonal entries simply tells us that we do not care what goes in there since we do not have self-loops in the graph.
 
The optimal $2$-TSP tour for $k=2$ salespeople is shown below.
  - Salesperson # 1: $0 \rightarrow 2 \rightarrow 1 \rightarrow 4 \rightarrow 0$.
  - Salesperson # 2: $0 \rightarrow 3 \rightarrow 0$.
  
The total cost of the edges traversed by the two salespeople equals $12$.

For $k=3$, the optimal $3-$ TSP tour is as shown below.
  - Salesperson # 1: $0 \rightarrow 1 \rightarrow 4$, 
  - Salesperson # 2: $0 \rightarrow 2$, 
  - Salesperson # 3: $0 \rightarrow 3$.

The total cost is $16$.

The objective of this problem is to formulate an ILP using the MTZ approach.

### Problem 1A (MTZ approach)

We will use the same ILP setup as in our notes (see the notes on Exact Approaches to TSP that includes the ILP encodings we will use in this problem).
  - Decision variables $x_{i,j}$ for $i \not= j$ denoting that the tour traverses the edge from $i$ to $j$.
  - Time stamps $t_1, \ldots, t_{n-1}$. The start/end vertex $0$ does not get a time stamp.
  
Modify the MTZ approach to incorporate the fact that $k$ salespeople are going to traverse the graph.

#### (A) Degree Constraints

What do the new degree constraints look like? Think about how many edges in the tour will need to enter/leave each vertex? Note that you may have to treat vertex $0$ differently from the other vertices of the graph.

Your answer below is not graded. However you are encouraged to write it down and check with the answers to select problems provided at the end.

YOUR ANSWER HERE

### (B) Time Stamp Constraints 

Formulate the time stamp constraints for the $k$-TSP problem. Think about how you would need to change them to eliminate subtour.

Your answer below is not graded. However you are encouraged to write it down and check with the answers to select problems provided at the end.


YOUR ANSWER HERE

### (C) Implement

Complete the implementation of the function `k_tsp_mtz_encoding(n, k, cost_matrix)` below. It follows the same input convention as the code supplied in the notes. The input `n` denotes the size of the graph with vertices labeled `0`,.., `n-1`, `k` is the number of salespeople, and `cost_matrix` is a list of lists wherein `cost_matrix[i][j]` is the edge cost to go from `i` to `j` for `i != j`. Your code must avoid accessing `cost_matrix[i][i]` to avoid bugs. These entries will be supplied as `None` in the test cases.

Your code must return a list `lst` that has exactly $k$ lists in it, wherein `lst[j]` represents the locations visited by the $j^{th}$ salesperson. 

For the example above, for $k=2$, your code must return 
~~~
[ [0, 2, 1, 4], [0, 3] ]
~~~
For the example above, for $k=3$, your code must return
~~~
[ [0, 1, 4], [0, 2], [0, 3] ]
~~~

In [1]:
!pip install --quiet geopy
!pip install --quiet cvxpy[GLPK]

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cp


def k_tsp_mtz_encoding(n, k, cost_matrix):
    # check inputs are OK
    assert 1 <= k < n
    assert len(cost_matrix) == n, f'Cost matrix is not {n}x{n}'
    assert all(len(cj) == n for cj in cost_matrix), f'Cost matrix is not {n}x{n}'
    #prob = LpProblem('kTSP', LpMinimize)
    # finish your implementation here
    # your code must return a list of k-lists [ [0, i1, ..., il], [0, j1,...,jl],...] wherein 
    # the ith entry in our list of lists represents the 
    # tour undertaken by the ith salesperson
    # your code here
    cost_matrix = [[0 if cost is None else cost for cost in row] for row in cost_matrix]
    m = k 
    C = np.zeros((n,n))
    for i in range(0, n):
        for j in range(0, n):
            C[i,j] = (cost_matrix[i][j])


    # Defining the variables
    X = cp.Variable(C.shape, boolean=True)
    u = cp.Variable(n, integer=True)
    ones = np.ones((n,1))

    # Defining the objective function
    objective = cp.Minimize(cp.sum(cp.multiply(C, X)))

    # Defining the constraints
    constraints = []
    constraints += [X[0,:] @ ones == m]
    constraints += [X[:,0] @ ones == m]
    constraints += [X[1:,:] @ ones == 1]
    constraints += [X[:,1:].T @ ones == 1]
    constraints += [cp.diag(X) == 0]
    constraints += [u[1:] >= 2]
    constraints += [u[1:] <= n]
    constraints += [u[0] == 1]

    for i in range(1, n):
        for j in range(1, n):
            if i != j:
                constraints += [ u[i] - u[j] + 1  <= (n - 1) * (1 - X[i, j]) ]

    # Solving the problem
    prob = cp.Problem(objective, constraints)
    prob.solve(verbose=False)

    # Transforming the solution to paths
    X_sol = np.argwhere(X.value==1)
    ruta = {}
    for i in range(0, m):
        ruta['Salesman_' + str(i+1)] = [0]
        j = i
        a = 10e10
        while a != 0:
            a = X_sol[j,1]
            ruta['Salesman_' + str(i+1)].append(a)
            j = np.where(X_sol[:,0] == a)
            j = j[0][0]
            a = j
        
    tours = []
    for i in ruta.keys():
        tours.append(ruta[i][:-1])
    
    return tours


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

(CVXPY) Nov 09 12:02:12 AM: Encountered unexpected exception importing solver SCS:
ImportError('numpy.core.multiarray failed to import')


Vertex 0:  𝑘
  edges leave and  𝑘
  edges enter.
Vertex 1, ..., n-1: 1 edge leaves and 1 edge enters (same as TSP).

In [43]:
from pulp import *

def k_tsp_mtz_encoding__(n, k, cost_matrix):
    assert len(cost_matrix) == n, f'Cost matrix is not {n}x{n}'
    assert all(len(cj) == n for cj in cost_matrix), f'Cost matrix is not {n}x{n}'

    # create our encoding variables
    binary_vars = [
        [LpVariable(f'x_{i}_{j}', cat='Binary') if i != j else None for j in range(n)]
        for i in range(n)
    ]

    # add time stamps for ranges 1 .. n (skip vertex 0 for timestamps)
    time_stamps = [LpVariable(f't_{j}', lowBound=0, upBound=n, cat='Continuous') for j in range(1, n)]

    # Create the problem
    prob = LpProblem(f'{k}-TSP-MTZ', LpMinimize)

    # Create and add the objective function
    objective_function = lpSum([lpSum([xij * cj if xij is not None else 0 for (xij, cj) in zip(brow, crow)]) for (brow, crow) in zip(binary_vars, cost_matrix)])
    prob += objective_function

    # Set 1: For Node[0], k edges leave and k edges enter
    prob += lpSum([xij for xij in binary_vars[0] if xij is not None]) == k
    prob += lpSum([binary_vars[j][0] for j in range(n) if j != 0]) == k

    # Set 2: For Node[1:n], 1 edge leaves and 1 edge enters
    for i in range(1, n):
        prob += lpSum([xij for xij in binary_vars[i] if xij is not None]) == 1
        prob += lpSum([binary_vars[j][i] for j in range(n) if j != i]) == 1

    # Add time stamp constraints
    for i in range(1, n):
        for j in range(1, n):
            if i == j:
                continue
            xij = binary_vars[i][j]
            ti = time_stamps[i - 1]
            tj = time_stamps[j - 1]
            prob += tj >= ti + xij - (1 - xij) * (n + 1)

    # Solve the problem
    status = prob.solve(PULP_CBC_CMD(msg=False))
    assert status == constants.LpStatusOptimal, f'Unexpected non-optimal status {status}'

    # Extract tours for each salesman
    tours = extract_k_tsp_tours(binary_vars, n, k)
    return tours

def extract_k_tsp_tours(binary_vars, n, k):
    tours = []

    # Initialize a list to track visited cities
    visited = [False] * n

    for _ in range(k):
        tour = []
        current_city = 0  # Start from city 0
        while True:
            tour.append(current_city)
            visited[current_city] = True

            # Find the next unvisited city in the current tour
            next_city = None
            for j in range(n):
                if not visited[j] and binary_vars[current_city][j].varValue == 1:
                    next_city = j
                    break

            if next_city is not None:
                current_city = next_city
            else:
                break  # Finished the current tour

        tours.append(tour)

        # Find the next unvisited city for the next tour
        for i in range(n):
            if not visited[i]:
                current_city = i
                break

    return tours




from random import uniform, randint

def create_cost(n):
    return [ [uniform(0, 5) if i != j else None for j in range(n)] for i in range(n)]

for trial in range(5):
    print(f'Trial # {trial}')
    n = randint(5, 11)
    k = randint(2, n//2)
    print(f' n= {n}, k={k}')
    cost_matrix = create_cost(n)
    print('cost_matrix = ')
    print(cost_matrix)
    all_tours = k_tsp_mtz_encoding__(n, k, cost_matrix)
    print(f'Your code returned tours: {all_tours}')
    assert len(all_tours) == k, f'k={k} must yield two tours -- your code returns {len(all_tours)} tours instead'

    tour_cost = 0
    for tour in all_tours:
        assert tour[0] == 0, 'Each salesperson tour must start from vertex 0'
        i = 0
        for j in tour[1:]:
            tour_cost += cost_matrix[i][j]
            i = j
        tour_cost += cost_matrix[i][0]

    print(f'Tour cost obtained by your code: {tour_cost}')
    #assert abs(tour_cost - 6) <= 0.001, f'Expected tour cost is 6, your code returned {tour_cost}'
    for i in range(1, n):
        is_in_tour = [ 1 if i in tour else 0 for tour in all_tours]
        assert sum(is_in_tour) == 1, f' vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect'
    print('------')
print('test passed: 15 points')

Trial # 0
 n= 11, k=2
cost_matrix = 
[[None, 3.5357762964200123, 3.055266666358569, 0.18190141350482525, 3.170400671977286, 4.144244977100512, 1.9460924489438565, 3.6105906506339864, 1.2699017174997835, 4.266261367730922, 4.6167113698311315], [2.69427495337856, None, 0.8334793215636144, 4.226985779005556, 4.763426992192485, 1.1291252952859336, 0.0071693063831740345, 3.3534020089964485, 3.5706551313913817, 3.58939644637797, 1.2388132178063915], [0.33712384774649884, 2.197841725586959, None, 1.4811818492989592, 2.1344966545630135, 3.2730672945875035, 2.473862222090016, 1.231040370596273, 4.449941824610722, 4.685199064140104, 4.352421044620501], [4.171025203484982, 3.049188696053593, 1.213761536722349, None, 4.2922127996933925, 3.9368067732128194, 1.8016449877250347, 4.579404562928975, 2.7430994492055176, 4.125147058346813, 1.9727190512043136], [0.7375927990773679, 3.762966631315967, 0.30299843607731414, 1.406953074970399, None, 2.790957288004478, 0.1183333044503182, 3.67396432919213, 4.2

In [2]:
import pulp as lp
import numpy as np

def k_tsp_mtz_encoding_(n, k, cost_matrix):
    assert 1 <= k < n
    assert len(cost_matrix) == n, f'Cost matrix is not {n}x{n}'
    assert all(len(cj) == n for cj in cost_matrix), f'Cost matrix is not {n}x{n}'

    # Create a PuLP problem and specify it as an integer linear programming (ILP) problem
    prob = lp.LpProblem("kTSP", lp.LpMinimize)

    # Create binary decision variables X[i][j] using LpVariable.dicts
    X = lp.LpVariable.dicts("X", [(i, j) for i in range(n) for j in range(n)], cat=lp.LpBinary)

    # Create integer variables u[i]
    u = lp.LpVariable.dicts("u", range(n), cat=lp.LpInteger)

    # Define the objective function using lp.lpSum
    prob += lp.lpSum(cost_matrix[i][j] * X[(i, j)] for i in range(n) for j in range(n))#, "Total Cost"

    # Define the constraints

    prob += lp.lpSum(X[(0, j)] for j in range(1, n)) == k# "Salespeople Start from City 0"
    prob += lp.lpSum(X[(i, 0)] for i in range(1, n)) == k# "Salespeople End at City 0"

    for i in range(1, n):
        prob += lp.lpSum(X[(i, j)] for j in range(n)) == 1 # f"Each City Visited Exactly Once - City {i}"

    for j in range(1, n):
        prob += lp.lpSum(X[(i, j)] for i in range(n)) == 1  # f"Each City Visited Exactly Once - City {j}"

    for i in range(n):
        prob += X[(i, i)] == 0# f"Diagonal Constraints - City {i}"

    for i in range(1, n):
        prob += u[i] >= 2# f"u >= 2 for City {i}"

    for i in range(1, n):
        prob += u[i] <= n# f"u <= n for City {i}"

    prob += u[0] == 1# "Starting City"

    for i in range(1, n):
        for j in range(1, n):
            if i != j:
                prob += u[i] - u[j] + 1 <= (n - 1) * (1 - X[(i, j)])# f"MTZ Constraints - ({i}, {j})"

    # Solve the problem
    prob.solve()

    # Reconstruct the solution
    X_sol = {(i, j): lp.value(X[(i, j)]) for i in range(n) for j in range(n)}
    tours = reconstruct_tours(X_sol, n, k)

    return tours

def reconstruct_tours(X_sol, n, k):
    # Reconstruct the solution to paths
    ruta = {}
    for i in range(k):
        ruta['Salesman_' + str(i+1)] = [0]
        j = i
        a = 10e10
        while a != 0:
            a = next((key[1] for key, value in X_sol.items() if key[0] == j and value == 1), None)
            if a is not None:
                ruta['Salesman_' + str(i+1)].append(a)
                j = a

    tours = []
    for i in ruta.keys():
        tours.append(ruta[i][:-1])

    return tours

# Example usage:
n = 5
k = 2
cost_matrix=[ [0,3,4,3,5],
             [1, 0, 2,4, 1],
             [2, 1, 0, 5, 4],
             [1, 1, 5, 0, 4],
             [2, 1, 3, 5, 0] ]
tours = k_tsp_mtz_encoding_(n, k, cost_matrix)
print(tours)


[[0, 2, 1, 4], [0, 4]]


In [3]:
cost_matrix=[ [None,3,4,3,5],
             [1, None, 2,4, 1],
             [2, 1, None, 5, 4],
             [1, 1, 5, None, 4],
             [2, 1, 3, 5, None] ]
n=5
k=2
all_tours = k_tsp_mtz_encoding(n, k, cost_matrix)
print(f'Your code returned tours: {all_tours}')
assert len(all_tours) == k, f'k={k} must yield two tours -- your code returns {len(all_tours)} tours instead'

tour_cost = 0
for tour in all_tours:
    assert tour[0] == 0, 'Each salesperson tour must start from vertex 0'
    i = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]

print(f'Tour cost obtained by your code: {tour_cost}')
assert abs(tour_cost - 12) <= 0.001, f'Expected tour cost is 12, your code returned {tour_cost}'
for i in range(1, n):
    is_in_tour = [ 1 if i in tour else 0 for tour in all_tours]
    assert sum(is_in_tour) == 1, f' vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect'

print('test passed: 3 points')

Your code returned tours: [[0, 2, 1, 4], [0, 3]]
Tour cost obtained by your code: 12
test passed: 3 points


In [4]:
import pulp

def k_tsp_mtz_encoding_pulp(n, k, cost_matrix):
    # check inputs are OK
    assert 1 <= k < n
    assert len(cost_matrix) == n, f'Cost matrix is not {n}x{n}'
    assert all(len(cj) == n for cj in cost_matrix), f'Cost matrix is not {n}x{n}'
    #prob = LpProblem('kTSP', LpMinimize)
    # finish your implementation here
    # your code must return a list of k-lists [ [0, i1, ..., il], [0, j1,...,jl],...] wherein 
    # the ith entry in our list of lists represents the 
    # tour undertaken by the ith salesperson
    # your code here
    cost_matrix = [[0 if cost is None else cost for cost in row] for row in cost_matrix]
    # Number of salespeople and nodes
    num_salespeople = k
    num_nodes = n

    # Create a binary variable to represent the assignment of nodes to salespeople
    x = pulp.LpVariable.dicts("x", ((i, j) for i in range(num_salespeople) for j in range(num_nodes)), cat='Binary')

    # Create the PuLP problem object
    problem = pulp.LpProblem("MultiSalespersonProblem", pulp.LpMinimize)

    # Objective function: minimize the total cost
    problem += pulp.lpSum(cost_matrix[i][j] * x[(i, j)] for i in range(num_salespeople) for j in range(num_nodes))

    # Constraints
    # Each node can be assigned to only one salesperson
    for j in range(num_nodes):
        problem += pulp.lpSum(x[(i, j)] for i in range(num_salespeople)) == 1

    # Each salesperson must be assigned to at least one node
    for i in range(num_salespeople):
        problem += pulp.lpSum(x[(i, j)] for j in range(num_nodes)) >= 1

    # Solve the problem
    problem.solve()

    tours = []
    # Print the results
    if pulp.LpStatus[problem.status] == "Optimal":
        
        for i in range(num_salespeople):
            assigned_nodes = [j for j in range(num_nodes) if x[(i, j)].varValue == 1 and j!=0]
            tour = [0]
            for j in assigned_nodes:
                tour.append(j)
            tours.append(tour)    
                
    else:
        print("No optimal solution found.")
        
        
    return tours
    

In [5]:
cost_matrix=[ [None,3,4,3,5],
             [1, None, 2,4, 1],
             [2, 1, None, 5, 4],
             [1, 1, 5, None, 4],
             [2, 1, 3, 5, None] ]
n=5
k=2
all_tours = k_tsp_mtz_encoding(n, k, cost_matrix)
print(f'Your code returned tours: {all_tours}')
assert len(all_tours) == k, f'k={k} must yield two tours -- your code returns {len(all_tours)} tours instead'

tour_cost = 0
for tour in all_tours:
    assert tour[0] == 0, 'Each salesperson tour must start from vertex 0'
    i = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]

print(f'Tour cost obtained by your code: {tour_cost}')
assert abs(tour_cost - 12) <= 0.001, f'Expected tour cost is 12, your code returned {tour_cost}'
for i in range(1, n):
    is_in_tour = [ 1 if i in tour else 0 for tour in all_tours]
    assert sum(is_in_tour) == 1, f' vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect'

print('test passed: 3 points')


Your code returned tours: [[0, 2, 1, 4], [0, 3]]
Tour cost obtained by your code: 12
test passed: 3 points


In [6]:
cost_matrix=[ [None,3,4,3,5],
             [1, None, 2,4, 1],
             [2, 1, None, 5, 4],
             [1, 1, 5, None, 4],
             [2, 1, 3, 5, None] ]
n=5
k=3
all_tours = k_tsp_mtz_encoding(n, k, cost_matrix)
print(f'Your code returned tours: {all_tours}')
assert len(all_tours) == k, f'k={k} must yield two tours -- your code returns {len(all_tours)} tours instead'

tour_cost = 0
for tour in all_tours:
    assert tour[0] == 0, 'Each salesperson tour must start from vertex 0'
    i = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]

print(f'Tour cost obtained by your code: {tour_cost}')
assert abs(tour_cost - 16) <= 0.001, f'Expected tour cost is 16, your code returned {tour_cost}'
for i in range(1, n):
    is_in_tour = [ 1 if i in tour else 0 for tour in all_tours]
    assert sum(is_in_tour) == 1, f' vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect'

print('test passed: 2 points')

Your code returned tours: [[0, 1, 4], [0, 2], [0, 3]]
Tour cost obtained by your code: 16
test passed: 2 points


In [7]:
cost_matrix = [ 
 [None, 1, 1, 1, 1, 1, 1, 1],
    [0, None, 1, 2, 1, 1, 1, 1],
    [1, 0, None, 1, 2, 2, 2, 1],
    [1, 2, 2, None, 0, 1, 2, 1],
    [1, 1, 1, 1, None, 1, 1, 1],
    [0,  1, 2, 1, 1, None, 1, 1],
    [1, 0,  1, 2, 2, 2,None, 1],
    [1, 2, 2, 0, 1, 2, 1, None],
]
n = 8
k = 2

all_tours = k_tsp_mtz_encoding(n, k, cost_matrix)
print(f'Your code returned tours: {all_tours}')
assert len(all_tours) == k, f'k={k} must yield two tours -- your code returns {len(all_tours)} tours instead'

tour_cost = 0
for tour in all_tours:
    assert tour[0] == 0, 'Each salesperson tour must start from vertex 0'
    i = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]

print(f'Tour cost obtained by your code: {tour_cost}')
assert abs(tour_cost - 4) <= 0.001, f'Expected tour cost is 4, your code returned {tour_cost}'
for i in range(1, n):
    is_in_tour = [ 1 if i in tour else 0 for tour in all_tours]
    assert sum(is_in_tour) == 1, f' vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect'

print('test passed: 3 points')

Your code returned tours: [[0, 2, 7, 3, 4, 5], [0, 6, 1]]
Tour cost obtained by your code: 4
test passed: 3 points


In [8]:
cost_matrix = [ 
 [None, 1, 1, 1, 1, 1, 1, 1],
    [0, None, 1, 2, 1, 1, 1, 1],
    [1, 0, None, 1, 2, 2, 2, 1],
    [1, 2, 2, None, 0, 1, 2, 1],
    [1, 1, 1, 1, None, 1, 1, 1],
    [0,  1, 2, 1, 1, None, 1, 1],
    [1, 0,  1, 2, 2, 2,None, 1],
    [1, 2, 2, 0, 1, 2, 1, None],
]
n = 8
k = 4

all_tours = k_tsp_mtz_encoding(n, k, cost_matrix)
print(f'Your code returned tours: {all_tours}')
assert len(all_tours) == k, f'k={k} must yield two tours -- your code returns {len(all_tours)} tours instead'

tour_cost = 0
for tour in all_tours:
    assert tour[0] == 0, 'Each salesperson tour must start from vertex 0'
    i = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]

print(f'Tour cost obtained by your code: {tour_cost}')
assert abs(tour_cost - 6) <= 0.001, f'Expected tour cost is 6, your code returned {tour_cost}'
for i in range(1, n):
    is_in_tour = [ 1 if i in tour else 0 for tour in all_tours]
    assert sum(is_in_tour) == 1, f' vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect'

print('test passed: 2 points')

Your code returned tours: [[0, 2, 1], [0, 5], [0, 6], [0, 7, 3, 4]]
Tour cost obtained by your code: 6
test passed: 2 points


In [9]:
from random import uniform, randint

def create_cost(n):
    return [ [uniform(0, 5) if i != j else None for j in range(n)] for i in range(n)]

for trial in range(5):
    print(f'Trial # {trial}')
    n = randint(5, 11)
    k = randint(2, n//2)
    print(f' n= {n}, k={k}')
    cost_matrix = create_cost(n)
    print('cost_matrix = ')
    print(cost_matrix)
    all_tours = k_tsp_mtz_encoding(n, k, cost_matrix)
    print(f'Your code returned tours: {all_tours}')
    assert len(all_tours) == k, f'k={k} must yield two tours -- your code returns {len(all_tours)} tours instead'

    tour_cost = 0
    for tour in all_tours:
        assert tour[0] == 0, 'Each salesperson tour must start from vertex 0'
        i = 0
        for j in tour[1:]:
            tour_cost += cost_matrix[i][j]
            i = j
        tour_cost += cost_matrix[i][0]

    print(f'Tour cost obtained by your code: {tour_cost}')
    #assert abs(tour_cost - 6) <= 0.001, f'Expected tour cost is 6, your code returned {tour_cost}'
    for i in range(1, n):
        is_in_tour = [ 1 if i in tour else 0 for tour in all_tours]
        assert sum(is_in_tour) == 1, f' vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect'
    print('------')
print('test passed: 15 points')

Trial # 0
 n= 10, k=4
cost_matrix = 
[[None, 1.7079423596593457, 2.8411820397114895, 0.8881128760283458, 4.661711968516048, 3.4780639628371386, 1.3667725232021277, 4.56227640251414, 3.6890591884160555, 1.2816010124983612], [3.9432588789583893, None, 4.307760348937544, 4.914334608099431, 0.5778805151059069, 2.969002803568968, 1.9479976835496133, 0.3668896946385908, 0.0037319937226559796, 2.744041399710002], [2.060417257160897, 4.064875591748964, None, 4.38002730127783, 2.6861865652520382, 3.286318000803303, 1.3921273611000629, 4.826942615136524, 2.2562321023594905, 4.044143307821585], [3.581023372073559, 1.4897134208138885, 0.16839912646111455, None, 0.9748982735055983, 1.658518762935881, 0.03906874406010674, 4.836740671417953, 3.545287886113307, 2.9263822055358943], [4.017543260216457, 0.14993341093382795, 2.290124921498986, 0.12172039493292042, None, 3.4663020321909666, 0.19391556027304802, 3.4722487804236573, 4.923483891181988, 1.5228427723547662], [2.7011062669536297, 0.444383723716

## Problem 1 B

Notice that in previous part, it happens that with $k=4$ salespeople, we actually get a worse cost than using $k=3$ people. You can try out a few examples to convince yourself as to why this happens. 

We wish to modify the problem to allow salespeople to idle. In other words, although we input $k$ salespeople, the tour we construct may involve $1 \leq l \leq k$ salespeople. 

Modify the ILP formulation from the previous problem to solve the problem of up to $k$ people rather than exactly $k$ salespeople. Note that we still require that every vertex be visited exactly once by some salesperson. 

Complete the implementation of the function `upto_k_tsp_mtz_encoding(n, k, cost_matrix)` below. It follows the same input convention as previous problem but note that we are now computing a tour with at most $k$ salespeople. In other words, not all salespeople need be employed in the tour.

Your code must return a list `lst` that has less than or equal to  $k$ lists, wherein `lst[j]` represents the locations visited by the $j^{th}$ salesperson. 

For Example-1 from the previous part above, for $k=2$ or $k=3$, your code must return 
~~~
[ [0, 3, 1, 4, 2] ]
~~~
As it turns out, in this example a single salesperson suffices to yield optimal cost.

In [10]:
!pip install --quiet geopy
!pip install --quiet cvxpy[GLPK]

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cp

def upk_tsp_mtz_encoding(n, k, cost_matrix):
    min_cost = float('inf')
    min_tours = []
    for l in range(k,0,-1):
        print("processing ... :", l)
        tours, tour_cost = k_tsp_mtz_encoding(n, l, cost_matrix)
        if tour_cost<min_cost:
            min_cost = tour_cost
            min_tours = tours 
    return min_tours        
    
def k_tsp_mtz_encoding(n, k, cost_matrix):
    # check inputs are OK
    assert 1 <= k < n
    assert len(cost_matrix) == n, f'Cost matrix is not {n}x{n}'
    assert all(len(cj) == n for cj in cost_matrix), f'Cost matrix is not {n}x{n}'
    #prob = LpProblem('kTSP', LpMinimize)
    # finish your implementation here
    # your code must return a list of k-lists [ [0, i1, ..., il], [0, j1,...,jl],...] wherein 
    # the ith entry in our list of lists represents the 
    # tour undertaken by the ith salesperson
    # your code here
    cost_matrix = [[0 if cost is None else cost for cost in row] for row in cost_matrix]
    m = k 
    C = np.zeros((n,n))
    for i in range(0, n):
        for j in range(0, n):
            C[i,j] = (cost_matrix[i][j])


    # Defining the variables
    X = cp.Variable(C.shape, boolean=True)
    u = cp.Variable(n, integer=True)
    ones = np.ones((n,1))

    # Defining the objective function
    objective = cp.Minimize(cp.sum(cp.multiply(C, X)))

    # Defining the constraints
    constraints = []
    constraints += [X[0,:] @ ones == m]
    constraints += [X[:,0] @ ones == m]  
    constraints += [X[1:,:] @ ones == 1]
    constraints += [X[:,1:].T @ ones == 1]
    constraints += [cp.diag(X) == 0]
    constraints += [u[1:] >= 2]
    constraints += [u[1:] <= n]
    constraints += [u[0] == 1]

    for i in range(1, n):
        for j in range(1, n):
            if i != j:
                constraints += [ u[i] - u[j] + 1  <= (n - 1) * (1 - X[i, j]) ]

    # Solving the problem
    prob = cp.Problem(objective, constraints)
    prob.solve(verbose=False)

    # Transforming the solution to paths
    X_sol = np.argwhere(X.value==1)
    ruta = {}
    for i in range(0, m):
        ruta['Salesman_' + str(i+1)] = [0]
        j = i
        a = 10e10
        while a != 0:
            a = X_sol[j,1]
            ruta['Salesman_' + str(i+1)].append(a)
            j = np.where(X_sol[:,0] == a)
            j = j[0][0]
            a = j
        
    tours = []
    for i in ruta.keys():
        tours.append(ruta[i][:-1])
        
    tour_cost = 0
    print(tours)
    for tour in tours:
        assert tour[0] == 0, 'Each salesperson tour must start from vertex 0'
        i = 0
        for j in tour[1:]:
            tour_cost += cost_matrix[i][j]
            i = j
        tour_cost += cost_matrix[i][0]    
    
    return tours, tour_cost


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [11]:
cost_matrix=[ [None,3,4,3,5],
             [1, None, 2,4, 1],
             [2, 1, None, 5, 4],
             [1, 1, 5, None, 4],
             [2, 1, 3, 5, None] ]
n=5
k=3
all_tours = upk_tsp_mtz_encoding(n, k, cost_matrix)
print(f'Your code returned tours: {all_tours}')
assert len(all_tours) <= k, f'<= {k} tours -- your code returns {len(all_tours)} tours instead'

tour_cost = 0
for tour in all_tours:
    assert tour[0] == 0, 'Each salesperson tour must start from vertex 0'
    i = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]

assert len(all_tours) == 1, f'In this example, just one salesperson is needed to optimally visit all vertices. Your code returns {len(all_tours)}'
print(f'Tour cost obtained by your code: {tour_cost}')
assert abs(tour_cost - 10) <= 0.001, f'Expected tour cost is 10, your code returned {tour_cost}'
for i in range(1, n):
    is_in_tour = [ 1 if i in tour else 0 for tour in all_tours]
    assert sum(is_in_tour) == 1, f' vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect'

print('test passed: 3 points')

processing ... : 3
[[0, 1, 4], [0, 2], [0, 3]]
processing ... : 2
[[0, 2, 1, 4], [0, 3]]
processing ... : 1
[[0, 3, 1, 4, 2]]
Your code returned tours: [[0, 3, 1, 4, 2]]
Tour cost obtained by your code: 10
test passed: 3 points


In [12]:
from random import uniform, randint

def create_cost(n):
    return [ [uniform(0, 5) if i != j else None for j in range(n)] for i in range(n)]

for trial in range(20):
    print(f'Trial # {trial}')
    n = randint(5, 11)
    k = randint(2, n//2)
    print(f' n= {n}, k={k}')
    cost_matrix = create_cost(n)
    print('cost_matrix = ')
    print(cost_matrix)
    all_tours = upk_tsp_mtz_encoding(n, k, cost_matrix)
    print(f'Your code returned tours: {all_tours}')
    assert len(all_tours) <= k, f'k={k} must yield two tours -- your code returns {len(all_tours)} tours instead'

    tour_cost = 0
    for tour in all_tours:
        assert tour[0] == 0, 'Each salesperson tour must start from vertex 0'
        i = 0
        for j in tour[1:]:
            tour_cost += cost_matrix[i][j]
            i = j
        tour_cost += cost_matrix[i][0]

    print(f'Tour cost obtained by your code: {tour_cost}')
    #assert abs(tour_cost - 6) <= 0.001, f'Expected tour cost is 6, your code returned {tour_cost}'
    for i in range(1, n):
        is_in_tour = [ 1 if i in tour else 0 for tour in all_tours]
        assert sum(is_in_tour) == 1, f' vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect'
    print('------')
print('test passed: 4 points')

Trial # 0
 n= 9, k=4
cost_matrix = 
[[None, 1.152986295753053, 4.938890001797233, 1.410038233925543, 2.8058145835090227, 1.8963515938403896, 1.8765552896545583, 4.671338739807174, 1.1659372320830137], [0.09102729350387695, None, 0.07796124775952706, 0.6732331981105977, 4.4362091200385585, 2.4553767849999684, 3.8656004239107453, 0.9884020951846034, 0.7042480307013288], [0.04768049819343145, 3.303836738939255, None, 0.4472561396531133, 0.1223433346756242, 2.9454936605139053, 2.704756609125851, 1.2652712614466037, 0.05651791127582351], [2.9804336266138516, 1.199669790612049, 1.8691697333213035, None, 0.037280206174538644, 3.7904702986974046, 3.1117615215402945, 0.9625016320180035, 1.7137953982116554], [0.7975423552398775, 0.8962827626661113, 1.4021478073023745, 4.527190644090273, None, 1.1464185210627187, 0.7975600673039818, 3.955628220655564, 2.7684752377933544], [4.51215643685067, 1.7317699506000916, 1.5325904026561399, 2.5519261897465606, 1.9926098500598965, None, 3.9685159470472446, 2

In [13]:
from pulp import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cp
def upto_k_tsp_mtz_encoding(n, k, cost_matrix):
    # check inputs are OK
    assert 1 <= k < n
    assert len(cost_matrix) == n, f'Cost matrix is not {n}x{n}'
    assert all(len(cj) == n for cj in cost_matrix), f'Cost matrix is not {n}x{n}'
    #prob = LpProblem('kTSP', LpMinimize)
    # finish your implementation here
    # your code must return a list of k-lists [ [0, i1, ..., il], [0, j1,...,jl],...] wherein 
    # the ith entry in our list of lists represents the 
    # tour undertaken by the ith salesperson
    # your code here
    min_cost = float('inf')
    min_tours = []
    for l in range(k,0,-1):
        print("processing ... :", l)
        tours, tour_cost = k_tsp_mtz_encoding(n, l, cost_matrix)
        if tour_cost<min_cost:
            min_cost = tour_cost
            min_tours = tours 
    return min_tours        
    
def k_tsp_mtz_encoding(n, k, cost_matrix):
    # check inputs are OK
    assert 1 <= k < n
    assert len(cost_matrix) == n, f'Cost matrix is not {n}x{n}'
    assert all(len(cj) == n for cj in cost_matrix), f'Cost matrix is not {n}x{n}'
    #prob = LpProblem('kTSP', LpMinimize)
    # finish your implementation here
    # your code must return a list of k-lists [ [0, i1, ..., il], [0, j1,...,jl],...] wherein 
    # the ith entry in our list of lists represents the 
    # tour undertaken by the ith salesperson
    # your code here
    cost_matrix = [[0 if cost is None else cost for cost in row] for row in cost_matrix]
    m = k 
    C = np.zeros((n,n))
    for i in range(0, n):
        for j in range(0, n):
            C[i,j] = (cost_matrix[i][j])


    # Defining the variables
    X = cp.Variable(C.shape, boolean=True)
    u = cp.Variable(n, integer=True)
    ones = np.ones((n,1))

    # Defining the objective function
    objective = cp.Minimize(cp.sum(cp.multiply(C, X)))

    # Defining the constraints
    constraints = []
    constraints += [X[0,:] @ ones == m]
    constraints += [X[:,0] @ ones == m]  
    constraints += [X[1:,:] @ ones == 1]
    constraints += [X[:,1:].T @ ones == 1]
    constraints += [cp.diag(X) == 0]
    constraints += [u[1:] >= 2]
    constraints += [u[1:] <= n]
    constraints += [u[0] == 1]

    for i in range(1, n):
        for j in range(1, n):
            if i != j:
                constraints += [ u[i] - u[j] + 1  <= (n - 1) * (1 - X[i, j]) ]

    # Solving the problem
    prob = cp.Problem(objective, constraints)
    prob.solve(verbose=False)

    # Transforming the solution to paths
    X_sol = np.argwhere(X.value==1)
    ruta = {}
    for i in range(0, m):
        ruta['Salesman_' + str(i+1)] = [0]
        j = i
        a = 10e10
        while a != 0:
            a = X_sol[j,1]
            ruta['Salesman_' + str(i+1)].append(a)
            j = np.where(X_sol[:,0] == a)
            j = j[0][0]
            a = j
        
    tours = []
    for i in ruta.keys():
        tours.append(ruta[i][:-1])
        
    tour_cost = 0
    print(tours)
    for tour in tours:
        assert tour[0] == 0, 'Each salesperson tour must start from vertex 0'
        i = 0
        for j in tour[1:]:
            tour_cost += cost_matrix[i][j]
            i = j
        tour_cost += cost_matrix[i][0]    
    
    return tours, tour_cost

In [14]:
cost_matrix=[ [None,3,4,3,5],
             [1, None, 2,4, 1],
             [2, 1, None, 5, 4],
             [1, 1, 5, None, 4],
             [2, 1, 3, 5, None] ]
n=5
k=3
all_tours = upto_k_tsp_mtz_encoding(n, k, cost_matrix)
print(f'Your code returned tours: {all_tours}')
assert len(all_tours) <= k, f'<= {k} tours -- your code returns {len(all_tours)} tours instead'

tour_cost = 0
for tour in all_tours:
    assert tour[0] == 0, 'Each salesperson tour must start from vertex 0'
    i = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]

assert len(all_tours) == 1, f'In this example, just one salesperson is needed to optimally visit all vertices. Your code returns {len(all_tours)}'
print(f'Tour cost obtained by your code: {tour_cost}')
assert abs(tour_cost - 10) <= 0.001, f'Expected tour cost is 10, your code returned {tour_cost}'
for i in range(1, n):
    is_in_tour = [ 1 if i in tour else 0 for tour in all_tours]
    assert sum(is_in_tour) == 1, f' vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect'

print('test passed: 3 points')

processing ... : 3
[[0, 1, 4], [0, 2], [0, 3]]
processing ... : 2
[[0, 2, 1, 4], [0, 3]]
processing ... : 1
[[0, 3, 1, 4, 2]]
Your code returned tours: [[0, 3, 1, 4, 2]]
Tour cost obtained by your code: 10
test passed: 3 points


In [15]:
# your code here
#raise NotImplementedError

In [16]:
from random import uniform, randint

def create_cost(n):
    return [ [uniform(0, 5) if i != j else None for j in range(n)] for i in range(n)]

for trial in range(20):
    print(f'Trial # {trial}')
    n = randint(5, 11)
    k = randint(2, n//2)
    print(f' n= {n}, k={k}')
    cost_matrix = create_cost(n)
    print('cost_matrix = ')
    print(cost_matrix)
    all_tours = upto_k_tsp_mtz_encoding(n, k, cost_matrix)
    print(f'Your code returned tours: {all_tours}')
    assert len(all_tours) <= k, f'k={k} must yield two tours -- your code returns {len(all_tours)} tours instead'

    tour_cost = 0
    for tour in all_tours:
        assert tour[0] == 0, 'Each salesperson tour must start from vertex 0'
        i = 0
        for j in tour[1:]:
            tour_cost += cost_matrix[i][j]
            i = j
        tour_cost += cost_matrix[i][0]

    print(f'Tour cost obtained by your code: {tour_cost}')
    #assert abs(tour_cost - 6) <= 0.001, f'Expected tour cost is 6, your code returned {tour_cost}'
    for i in range(1, n):
        is_in_tour = [ 1 if i in tour else 0 for tour in all_tours]
        assert sum(is_in_tour) == 1, f' vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect'
    print('------')
print('test passed: 4 points')

Trial # 0
 n= 8, k=3
cost_matrix = 
[[None, 0.7065684147341739, 4.742512011453669, 2.0802117050486633, 0.6923441841670275, 4.388636859350096, 2.6113043112565264, 4.984839637535487], [0.08154400179732013, None, 0.4545693051994737, 1.7174830574127515, 4.128729651445308, 3.4516122826428925, 3.7936248036063427, 3.65197094686733], [4.552598074408488, 0.3768565792463002, None, 0.3023576157584418, 2.096231501707662, 2.3628601111557384, 0.972116686507693, 1.000249050739942], [2.526203639913483, 1.2301645241302532, 2.4507255483601624, None, 3.608509840940694, 4.317842416728441, 2.2257928077763545, 3.304073144914779], [1.0784755232522747, 4.184861752404033, 3.8718958852498253, 4.6214409842081485, None, 1.3028187957970683, 0.020571752267218857, 1.4115400806480904], [1.5298060947505205, 3.3994230752555583, 0.7618529585010697, 4.323606217162024, 1.9397500638494014, None, 3.781572416418129, 2.6649585440121237], [4.526456470061625, 3.9864331696897084, 0.14574673172222496, 4.451545908396039, 3.3875327

## Problem 2 (10 points)

We noted the use of Christofides algorithm for metric TSP. We noted that for non-metric TSPs it does not work. 
In fact, the shortcutting used in Christofides algorithm can be _arbitrarily_ bad for a TSP that is symmetric but fails to be a metric TSP.

In this example, we would like you to frame a symmetric TSP instance ($C_{ij} = C_{ji}$) with $5$ vertices wherein the algorithm obtained by "shortcutting" the minimum spanning tree (MST), that would be a 2-factor approximation for metric TSP, yields an answer that can be quite "far off" from the optimal solution.

Enter a __symmetric__ cost-matrix for the TSP below as a 5x5 matrix as a list of lists following convention in our notes. Do not worry about the diagonal entries.

Ensure that the optimal answer is at least $10^6$ times smaller than that obtained by the TSP-based approximation. We will test your answer by running the TSP with shortcutting algorithm.

__Hint:__ Force the edges $(0,1), (1,2), (2,3)$ and $(3,4)$ to be the minimum spanning tree. But make the weight of the edge form $4$ back to $0$ very high.


__Note:__ this problem is tricky and requires you to be very familiar with how Christofides algorithm works. It may be wise to attempt the remaining problems first before this one.


In [ ]:
# Write down the cost matrix as a list of lists. Remember, we want an example with $5$ vertices
# cost_matrix = [[..], [...],[...]]
# your code here
raise NotImplementedError

In [ ]:
# check that the cost matrix is symmetric a
assert len(cost_matrix) == 5, f'Cost matrix must have 5 rows. Yours has {len(cost_matrix)} rows'
assert all(len(cj) == 5 for cj in cost_matrix), f'Each row of the cost matrix must have 5 entries.'
for i in range(5):
    for j in range(i):
        assert cost_matrix[i][j] == cost_matrix[j][i], f'Cost matrix fails to be symmetric at entries {(i,j)} and {(j,i)}'
print('Structure of your cost matrix looks OK (3 points).')

Please ensure that you run the two cells below or else, your tests will fail.

In [ ]:
# MST based tsp approximation
import networkx as nx

# This code implements the simple MST based shortcutting approach that would yield factor of 2
# approximation for metric TSPs.
def minimum_spanning_tree_tsp(n, cost_matrix):
    G = nx.Graph()
    for i in range(n):
        for j in range(i):
            G.add_edge(i, j, weight=cost_matrix[i][j])
    T = nx.minimum_spanning_tree(G)
    print(f'MST for your graph has the edges {T.edges}')
    mst_cost = 0
    mst_dict = {} # store mst as a dictionary
    for (i,j) in T.edges:
        mst_cost += cost_matrix[i][j]
        if i in mst_dict:
            mst_dict[i].append(j)
        else:
            mst_dict[i] = [j]
        if j in mst_dict:
            mst_dict[j].append(i)
        else:
            mst_dict[j] = [i]
    print(f'MST cost: {mst_cost}')
    print(mst_dict)
    # Let's form a tour with short cutting
    def traverse_mst(tour_so_far, cur_node):
        assert cur_node in mst_dict
        next_nodes = mst_dict[cur_node]
        for j in next_nodes:
            if j in tour_so_far:
                continue
            tour_so_far.append(j)
            traverse_mst(tour_so_far, j)
        return
    tour = [0]
    traverse_mst(tour, 0)
    i = 0
    tour_cost = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]
    return tour, tour_cost

In [ ]:
# optimal TSP tour taken from our notes using MTZ encoding
from pulp import *

def mtz_encoding_tsp(n, cost_matrix):
    assert len(cost_matrix) == n, f'Cost matrix is not {n}x{n}'
    assert all(len(cj) == n for cj in cost_matrix), f'Cost matrix is not {n}x{n}'
    # create our encoding variables
    binary_vars = [ # add a binary variable x_{ij} if i not = j else simply add None
        [ LpVariable(f'x_{i}_{j}', cat='Binary') if i != j else None for j in range(n)] 
        for i in range(n) ]
    # add time stamps for ranges 1 .. n (skip vertex 0 for timestamps)
    time_stamps = [LpVariable(f't_{j}', lowBound=0, upBound=n, cat='Continuous') for j in range(1, n)]
    # create the problem
    prob = LpProblem('TSP-MTZ', LpMinimize)
    # create add the objective function 
    objective_function = lpSum( [ lpSum([xij*cj if xij != None else 0 for (xij, cj) in zip(brow, crow) ])
                           for (brow, crow) in zip(binary_vars, cost_matrix)] )
    
    prob += objective_function 
    
    # add the degree constraints
    for i in range(n):
        # Exactly one leaving variable
        prob += lpSum([xj for xj in binary_vars[i] if xj != None]) == 1
        # Exactly one entering
        prob += lpSum([binary_vars[j][i] for j in range(n) if j != i]) == 1
    # add time stamp constraints
    for i in range(1,n):
        for j in range(1, n):
            if i == j: 
                continue
            xij = binary_vars[i][j]
            ti = time_stamps[i-1]
            tj = time_stamps[j -1]
            prob += tj >= ti + xij - (1-xij)*(n+1) # add the constraint
    # Done: solve the problem
    status = prob.solve(PULP_CBC_CMD(msg=False)) # turn off messages
    assert status == constants.LpStatusOptimal, f'Unexpected non-optimal status {status}'
    # Extract the tour
    tour = [0]
    tour_cost = 0
    while len(tour) < n:
        i = tour[-1]
        # find all indices j such that x_ij >= 0.999 
        sols = [j for (j, xij) in enumerate(binary_vars[i]) if xij != None and xij.varValue >= 0.999]
        assert len(sols) == 1, f'{sols}' # there better be just one such vertex or something has gone quite wrong
        j = sols[0] # extract the lone solutio 
        tour_cost = tour_cost + cost_matrix[i][j] # add to the tour cost
        tour.append(j) # append to the tour
        assert j != 0
    i = tour[-1]
    tour_cost = tour_cost + cost_matrix[i][0]
    return tour, tour_cost
        

In [ ]:
tour, tour_cost = minimum_spanning_tree_tsp(5, cost_matrix)
print(f'MST approximation yields tour is {tour} with cost {tour_cost}')
opt_tour, opt_tour_cost = mtz_encoding_tsp(5, cost_matrix)
print(f'Optimal tour is {opt_tour} with cost {opt_tour_cost}')

assert tour_cost/opt_tour_cost >= 1E+06, 'The TSP + shortcutting tour must be at least 10^6 times costlier than optimum. In your case, the ratio is {tour_cost/opt_tour_cost}'
print('Test passed: 7 points')

## Problem 3

In this problem, we wish to solve TSP with additional constraints. Suppose we are given a TSP instance in the form of a $n\times n$ matrix $C$ representing a complete graph. 

We wish to solve a TSP but with additional constraints specified as a list $[(i_0, j_0), \ldots, (i_k, j_k)]$ wherein each pair $(i_l, j_l)$ in the list specifies that vertex $i_l$ must be visited in the tour before vertex $j_l$. Assume that the tour starts/ends at vertex $0$ and none of the vertices in the constraint list is $0$. I.e, $i_l\not= 0, j_l \not= 0$ for all $0 \leq l \leq k$.

Modify one of the ILP encodings we have presented to solve TSP with extra constraints. Implement your solution in the function `tsp_with_extra_constraints(n, cost_matrix, constr_list)` where the extra argument `constr_list` is a list of pairs `[(i0,j0),...., (ik, jk)]` that specify for each pair `(il,jl)` that vertex `il` must be visited before `jl`. Assume that the problem is feasible (no need to handle infeasible instances). 
Your code should output the optimal tour as a list.

## Example

Consider again the graph with $5$ nodes and the following cost matrix from problem 1:

$$ \begin{array}{c|ccccc}
  \text{Vertices} & 0 & 1 & 2 & 3 & 4 \\ 
   \hline
 0 & - & 3 & 4 & 3 & 5 \\ 
 1 & 1 & - & 2 & 4 & 1 \\ 
 2 & 2 & 1 & - & 5 & 4 \\ 
 3 & 1 & 1 & 5 & - & 4 \\ 
 4 & 2 & 1 & 3 & 5 & - \\ 
 \end{array}$$
 
The optimal TSP tour will be $[0, 3, 1, 4, 2]$ with total cost $10$.

Suppose we added the constraints $[(4, 3), (1, 2)]$ we note that the tour satisfies the constraint $(1, 2)$ since it visits vertex $1$ before vertex $2$ but it unfortunately, $(4,3)$ is violated since vetex $3$ is visited before $4$ in the tour.


In [ ]:
def tsp_with_extra_constraints(n, cost_matrix, constraints):
    assert len(cost_matrix) == n, f'Cost matrix is not {n}x{n}'
    assert all(len(cj) == n for cj in cost_matrix), f'Cost matrix is not {n}x{n}'
    assert all( 1 <= i < n and 1 <= j < n and i != j for (i,j) in constraints)
    # TODO: encode the problem in pulp (a) decision variables; (b) constraints; (c) objective; (d) solve and extract
    #       solution. This is going to be very close to the MTZ encoding that we have presented in our notes. You can use
    #       our code as a starting point.
    # your code here
    raise NotImplementedError
       

In [ ]:
cost_matrix=[ [None,3,4,3,5],
             [1, None, 2,4, 1],
             [2, 1, None, 5, 4],
             [1, 1, 5, None, 4],
             [2, 1, 3, 5, None] ]
n=5
constraints = [(3,4),(1,2)]
tour = tsp_with_extra_constraints(n, cost_matrix, constraints)
i = 0
tour_cost = 0
for j in tour[1:]:
    tour_cost += cost_matrix[i][j]
    i = j
tour_cost += cost_matrix[i][0]
print(f'Tour:{tour}')
print(f'Cost of your tour: {tour_cost}')
assert abs(tour_cost-10) <= 0.001, 'Expected cost was 10'
for i in range(n):
    num = sum([1 if j == i else 0 for j in tour])
    assert  num == 1, f'Vertex {i} repeats {num} times in tour'
for (i, j) in constraints:
    assert tour.index(i) < tour.index(j), f'Tour does not respect constraint {(i,j)}'
print('Test Passed (3 points)')

In [ ]:
cost_matrix=[ [None,3,4,3,5],
             [1, None, 2,4, 1],
             [2, 1, None, 5, 4],
             [1, 1, 5, None, 4],
             [2, 1, 3, 5, None] ]
n=5
constraints = [(4,3),(1,2)]
tour = tsp_with_extra_constraints(n, cost_matrix, constraints)
i = 0
tour_cost = 0
for j in tour[1:]:
    tour_cost += cost_matrix[i][j]
    i = j
tour_cost += cost_matrix[i][0]
print(f'Tour:{tour}')
print(f'Cost of your tour: {tour_cost}')
assert abs(tour_cost-13) <= 0.001, 'Expected cost was 13'
for i in range(n):
    num = sum([1 if j == i else 0 for j in tour])
    assert  num == 1, f'Vertex {i} repeats {num} times in tour'
for (i, j) in constraints:
    assert tour.index(i) < tour.index(j), f'Tour does not respect constraint {(i,j)}'
print('Test Passed (3 points)')

In [ ]:
from random import uniform, randint

def create_cost(n):
    return [ [uniform(0, 5) if i != j else None for j in range(n)] for i in range(n)]

for trial in range(20):
    print(f'Trial # {trial}')
    n = randint(6, 11)
    cost_matrix = create_cost(n)
    constraints = [(1, 3), (4, 2), (n-1, 1), (n-2, 2)]
    tour = tsp_with_extra_constraints(n, cost_matrix, constraints)
    i = 0
    tour_cost = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]
    print(f'Tour:{tour}')
    print(f'Cost of your tour: {tour_cost}')
    for i in range(n):
        num = sum([1 if j == i else 0 for j in tour])
        assert  num == 1, f'Vertex {i} repeats {num} times in tour'
    for (i, j) in constraints:
        assert tour.index(i) < tour.index(j), f'Tour does not respect constraint {(i,j)}'
print('Test Passed (10 points)')

## Answers to Select Problems

### 1A part A

- Vertex 0: $k$ edges leave and $k$ edges enter.
- Vertex 1, ..., n-1: 1 edge leaves and 1 edge enters (same as TSP).

### 1A part B

This is a trick question. There is no need to change any of the time stamp related constraints.


## That's All Folks!